# Import Libraries

In [1]:
import pandas as pd
import pathlib as pl
import plotly.express as px
import folium

In [2]:
def create_output_directories(base_path):
    """
    Creates necessary output directories for data img and notebooks.
    """
    
    folders_to_create = ['data','img','notebooks']
    
    list_of_folders = []
    
    for main_folder in folders_to_create:
            folder_path = base_path.joinpath(main_folder)
            folder_path.mkdir(parents=True, exist_ok=True)
            list_of_folders.append(folder_path)
    
    return list_of_folders

In [3]:
def create_data_directories(base_path):
    """
    Creates necessary output directories for data img and notebooks.
    """
    
    folders_to_create = ['raw','processed']
    
    list_of_folders = []
    
    for main_folder in folders_to_create:
            folder_path = base_path.joinpath(main_folder)
            folder_path.mkdir(parents=True, exist_ok=True)
            list_of_folders.append(folder_path)
    
    return list_of_folders

In [4]:
current_path    = pl.Path.cwd().parent
output_folders  = create_output_directories(current_path)
data_folders    = create_data_directories(output_folders[0])
docs_list       = list(data_folders[0].glob('*.xlsx'))

# Import the dataset

In [5]:
df              = pd.read_excel(docs_list[0], sheet_name='Sheet0')
df              = df.drop(0, axis=0)


d:\Repositorios\Vanessa - subjetividades\subjetividades-politicas\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Codes 

In [6]:
JenniferCali = df[df['Sex_3_TEXT'] == 'JenniferCali(00008909547)'].index
df.loc[JenniferCali, 'Q78'] = '00008909547-JAVCALI'

codigo1  = df[df['Q69'] == '00008909547'].index
df.loc[codigo1, 'Q78'] = '00008909547-JAVCALI'
df.loc[codigo1, 'Q69'] = 'CALI'

In [7]:
# columns to drop
df              = df.drop(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress','Duration (in seconds)', 'Finished', 'RecordedDate','RecipientLastName','RecipientFirstName', 'RecipientEmail','ExternalReference','DistributionChannel', 'UserLanguage', 'Q66','Sex_3_TEXT','Region_3_TEXT'], axis=1)
full_size_df    = df.shape[0]
df.head()

,ResponseId,LocationLatitude,LocationLongitude,Sex,Age,EDuc,Etnia,Q69,Q70,Identity Fusion_1,...,Q78_2,Q78_3,Q78_4,Deshum_1,Deshum_2,Deshum_3,Q63,Q64,Q64_5_TEXT,Q78
1,R_1mIsUivDPG5uKUQ,6.2529,-75.5646,Un hombre,52,Posgrado,Ninguna,NaN,NaN,Ni de acuerdo ni en desacuerdo,...,NaN,NaN,NaN,74,74,100,Gustavo Petro,Gustavo Petro,NaN,NaN
2,R_1ApG86q8OuiAnmh,4.6913,-74.032,Un hombre,24,Universitario,Ninguna,NaN,NaN,Algo en desacuerdo,...,NaN,NaN,NaN,100,100,100,Sergio Fajardo,Sergio Fajardo,NaN,NaN
3,R_2yeGOP3ujdRCNoW,3.4129,-76.5191,Una mujer,25,Universitario,Ninguna,NaN,NaN,Ni de acuerdo ni en desacuerdo,...,NaN,NaN,NaN,40,80,90,Gustavo Petro,Gustavo Petro,NaN,NaN
4,R_Z4C1CcmO4uRFfkB,6.2529,-75.5646,Una mujer,23,Secundaria,Ninguna,NaN,NaN,De acuerdo,...,NaN,NaN,NaN,65,65,50,Gustavo Petro,No votaría,NaN,NaN
5,R_3Wa3NbYxLI2Do65,6.2529,-75.5646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,20,100,46,NaN,NaN,NaN,NaN


# Data Preprocessing

In [8]:
def create_bar_chart(df, column_name, file_name, plot_title,output_folder):
    unique_values = df[column_name].value_counts()

    # Create a bar chart using Plotly Express
    fig = px.bar(x=unique_values.index, y=unique_values.values)

    # Update layout if needed
    fig.update_layout(
        title='Bar Chart of '+ plot_title,
        xaxis_title="Unique Values",
        yaxis_title="Count"
    )

    # Show the plot
    fig.show()

    # Save the plot as an HTML file
    fig.write_html(output_folder.joinpath('bar_chart_'+file_name + '.html'))


In [9]:
def temp_value_counts(df, column):
    """
    This function returns the value counts of a column in a dataframe.
    """
    return print(df[column].value_counts())

# First filter: Age

In [10]:
df['Age']   = df['Age'].str.replace(r'(años|\. Años|AÑOS|de|Y i)', '', regex=True)
df          = df.dropna(subset=['Age']) # Drop rows with NaN values in Age column
df['Age']   = pd.to_numeric(df['Age'], errors='coerce')
df          = df[(df['Age'] >= 30) & (df['Age'] <= 64)] # Set a fitler for age between 30 and 64
df.head()

,ResponseId,LocationLatitude,LocationLongitude,Sex,Age,EDuc,Etnia,Q69,Q70,Identity Fusion_1,...,Q78_2,Q78_3,Q78_4,Deshum_1,Deshum_2,Deshum_3,Q63,Q64,Q64_5_TEXT,Q78
1,R_1mIsUivDPG5uKUQ,6.2529,-75.5646,Un hombre,52.0,Posgrado,Ninguna,NaN,NaN,Ni de acuerdo ni en desacuerdo,...,NaN,NaN,NaN,74,74,100,Gustavo Petro,Gustavo Petro,NaN,NaN
10,R_2E9RYbrROGCF1wv,NaN,NaN,Una mujer,38.0,Posgrado,Mestizo,NaN,NaN,Fuertemente de acuerdo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,R_71YJr70DUHCqpCV,6.2529,-75.5646,Un hombre,53.0,Posgrado,Mestizo,NaN,NaN,Ni de acuerdo ni en desacuerdo,...,NaN,NaN,NaN,100,100,100,Gustavo Petro,A Otro. Cuál?,Iván Cepeda,NaN
19,R_1mqHVaPGjnMWOLe,6.2529,-75.5646,Una mujer,30.0,Posgrado,Mestizo,NaN,NaN,En desacuerdo,...,NaN,NaN,NaN,92,96,98,Gustavo Petro,Gustavo Petro,NaN,NaN
24,R_27swIqQuXrpuqsN,6.2529,-75.5646,Un hombre,53.0,Posgrado,Mestizo,NaN,NaN,Ni de acuerdo ni en desacuerdo,...,NaN,NaN,NaN,26,98,100,No voté,No votaría,NaN,NaN


## Loss information when filtering by age

In [11]:
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')
print(f'Loss information: {100*(1-df.shape[0]/full_size_df)}%')

Number of rows: 350
Number of columns: 140
Loss information: 70.53872053872054%


## Column Q70: Department

In [12]:
df['Q70'] = df['Q70'].str.strip().str.upper()
replace_dict = {
    'VALLE'                 : 'VALLE DEL CAUCA',
    'VALLE DE CAUCA'        : 'VALLE DEL CAUCA',
    'VALLE CAUCA'           : 'VALLE DEL CAUCA',
    'CALI'                  : 'VALLE DEL CAUCA',
    'CALLE DEL CAUCA'       : 'VALLE DEL CAUCA',
    'DEPTO VALLE DEL CAUCA' : 'VALLE DEL CAUCA',
    'VALLE DWL CAUCA'       : 'VALLE DEL CAUCA',
    'VALE DEL CAUCA'        : 'VALLE DEL CAUCA',
    'JAMUNDI'               : 'VALLE DEL CAUCA',
    'VALLE DE L CAUCA'      : 'VALLE DEL CAUCA',
    'ANTIOQUÍA'             : 'ANTIOQUIA',
    'ANTIQUIA'              : 'ANTIOQUIA',
    'ENVIGADO'              : 'ANTIOQUIA',
    'MEDELLIN'              : 'ANTIOQUIA',
    'ANTIOGUIA'             : 'ANTIOQUIA',
    'BOGOTÁ'                : 'BOGOTA',
    'BOGOTÁ D.C.'           : 'BOGOTA',
    'BOGOTÁ D.C'            : 'BOGOTA',
    'DISTRITO CAPITAL'      : 'BOGOTA',
    'CÉSAR'                 : 'CESAR',
    'CESÁR'                 : 'CESAR',
    'C ESAR'                : 'CESAR',
    'CASAR'                 : 'CESAR',
    'VALLEDUPAR'            : 'CESAR',
    'ATLÁNTICO'             : 'ATLANTICO',
    'BUCARAMANGA'           : 'SANTANDER',
    'SNDER'                 : 'SANTANDER',
    'FLORIDA BLANCA'        : 'SANTANDER',
    'SAMTANDER'             : 'SANTANDER',
    'BOYACÁ'                : 'BOYACA', 
    'RIOHACHA'              : 'LA GUAJIRA',
    'GUAJIRA'               : 'LA GUAJIRA',
    'DEPARTAMENTO DEL CAUCA': 'CAUCA',
    'BOLÍVAR'               : 'BOLIVAR',
    'CHOCÓ'                 : 'CHOCO',
}
df['Q70'] = df['Q70'].replace(replace_dict)
temp_value_counts(df, 'Q70')
create_bar_chart(df, 'Q70', 'departments', 'Departments', output_folders[1])

Q70
VALLE DEL CAUCA       132
ANTIOQUIA             100
SANTANDER              43
CESAR                  40
CUNDINAMARCA           10
BOLIVAR                 4
BOGOTA                  3
NARIÑO                  3
CAUCA                   2
CHOCO                   1
C JA HHJGG              1
LA GUAJIRA              1
MAGDALENA               1
NORTE DE SANTANDER      1
BOYACA                  1
Name: count, dtype: int64


## Column Q69: Cities

In [13]:
df['Q69'] = df['Q69'].str.strip().str.upper()
replace_dict = {
    'SANTIAGO DE CALI'      : 'CALI',
    'CAÑASGORDAS'           : 'CALI',
    'CALI VALLE'            : 'CALI',
    'CALI VALLE DEL CAUCA'  : 'CALI',
    'EN SANTIAGO SDE CALL'  : 'CALI',
    'MEDELLÍN'              : 'MEDELLIN',
    'MEDELIN'               : 'MEDELLIN',
    'ENVIGADO'              : 'MEDELLIN',
    'ENVIGAADO'             : 'MEDELLIN',
    'RIONEGRO'              : 'MEDELLIN',
    'SABANETA'              : 'MEDELLIN',
    'BELLO'                 : 'MEDELLIN',
    'GIRARDOT'              : 'MEDELLIN',
    'ITAGUI'                : 'MEDELLIN',
    'ITAGÜÍ'                : 'MEDELLIN',
    'BOGOTÁ'                : 'BOGOTA',
    'BOGOTÁ D.C'            : 'BOGOTA',
    'BOGOTÁ D.C.'           : 'BOGOTA',
    'JAMUNDÍ'               : 'JAMUNDI',
    'VLLLEDUPR'             : 'VALLEDUPAR',
    'VALEDUPAR'             : 'VALLEDUPAR',
    'FLORIDA BLANCA'        : 'FLORIDABLANCA',
    'CÚCUTA'                : 'CUCUTA',
    'QUIBDÓ'                : 'QUIBDO',
}
df['Q69'] = df['Q69'].replace(replace_dict)

temp_value_counts(df, 'Q69')
# create_bar_chart(df, 'Q69', 'cities', 'Cities', output_folders[1])

Q69
CALI                      118
MEDELLIN                   90
VALLEDUPAR                 37
BUCARAMANGA                22
FLORIDABLANCA              14
BOGOTA                     10
JAMUNDI                     8
PIEDECUESTA                 5
BUENAVENTURA                4
PALMIRA                     4
CARTAGENA                   4
PASTO                       2
CALDAS                      1
PIENDAMO                    1
MARINILLA                   1
VALLE DEL CAUCA             1
YUMBO                       1
SAN JUAN DE PASTO           1
GUARNE                      1
TRUJILLO                    1
CAUCASIA                    1
CHIMICHAGUA                 1
SANTANDER DE QUILICHAO      1
EL COPEY                    1
TUNJA                       1
CUCUTA                      1
SANTA MARTA                 1
LA CEJA                     1
BOSCONIA                    1
LA PINTADA                  1
RIOHACHA                    1
SOCORRO                     1
GIRON                       1
SAN VI

## 2nd filter: Cities

In [14]:
cali_df = df[df['Q69'] == 'CALI']
cali_df.to_csv(data_folders[1].joinpath('cali.csv'))
cali_df.to_excel(data_folders[1].joinpath('cali.xlsx'))
cali_df.head()

,ResponseId,LocationLatitude,LocationLongitude,Sex,Age,EDuc,Etnia,Q69,Q70,Identity Fusion_1,...,Q78_2,Q78_3,Q78_4,Deshum_1,Deshum_2,Deshum_3,Q63,Q64,Q64_5_TEXT,Q78
88,R_1kLcWuTC7wCd3kf,3.4129,-76.5191,Una mujer,35.0,Universitario,Ninguna,CALI,VALLE DEL CAUCA,Algo en desacuerdo,...,Completamente de acuerdo,De acuerdo,Moderadamente de acuerdo,71,94,48,Gustavo Petro,Gustavo Petro,NaN,NaN
113,R_YSqy9LKNrvJCUHT,3.4129,-76.5191,Un hombre,46.0,Posgrado,Mestizo,CALI,VALLE DEL CAUCA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,R_3lS6rYyZtlfoUM0,3.4129,-76.5191,Una mujer,55.0,Técnico o Tecnológico,Afrodescendiente,CALI,VALLE DEL CAUCA,De acuerdo,...,NaN,NaN,NaN,18,18,15,No voté,No votaría,NaN,NaN
255,R_2zoHnCAn6RAEMZ3,4.6913,-74.032,Una mujer,34.0,Universitario,Mestizo,CALI,VALLE DEL CAUCA,De acuerdo,...,En desacuerdo,En desacuerdo,En desacuerdo,0,0,0,No quiero responder,Sergio Fajardo,NaN,NaN
256,R_1dKXNeYwm48UtUJ,3.4129,-76.5191,Una mujer,55.0,Técnico o Tecnológico,Afrodescendiente,CALI,VALLE DEL CAUCA,De acuerdo,...,Moderadamente de acuerdo,Moderadamente de acuerdo,Para nada de acuerdo,17,19,24,No voté,No votaría,NaN,30000121119UsbCali


In [15]:
print(f'Number of rows in Cali: {cali_df.shape[0]}')
print(f'Number of columns in Cali: {cali_df.shape[1]}')

Number of rows in Cali: 118
Number of columns in Cali: 140


## Column Sex

In [16]:
cali_df['Sex'] = cali_df['Sex'].str.strip().str.upper()
replace_dict = {
    'UNA MUJER' : 'MUJER',
    'UN HOMBRE' : 'HOMBRE'
}
cali_df['Sex'] = cali_df['Sex'].replace(replace_dict)
temp_value_counts(cali_df, 'Sex')
create_bar_chart(cali_df, 'Sex', 'sex', 'Sex', output_folders[1])

Sex
MUJER     66
HOMBRE    49
OTRO       1
Name: count, dtype: int64


C:\Users\hamso\AppData\Local\Temp\ipykernel_22136\2714689283.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hamso\AppData\Local\Temp\ipykernel_22136\2714689283.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Column EDuc: Education

In [17]:
temp_value_counts(cali_df, 'EDuc')
create_bar_chart(cali_df, 'EDuc', 'education', 'Education', output_folders[1])

EDuc
Universitario            39
Posgrado                 30
Técnico o Tecnológico    27
Secundaria               19
Primaria                  1
Prescolar                 1
Name: count, dtype: int64


## Column Etnia: Ethnicity

In [18]:
temp_value_counts(cali_df, 'Etnia')
create_bar_chart(cali_df, 'Etnia', 'ethnicity', 'Ethnicity', output_folders[1])

Etnia
Mestizo             39
Ninguna             34
Blanco              31
Afrodescendiente     9
Indígena             2
Name: count, dtype: int64


## Column Age

In [19]:
temp_value_counts(cali_df, 'Age')
create_bar_chart(cali_df, 'Age', 'age', 'Age', output_folders[1])

Age
42.0    9
47.0    7
49.0    6
43.0    6
35.0    6
44.0    5
53.0    5
45.0    4
36.0    4
34.0    4
39.0    4
41.0    4
30.0    4
40.0    4
32.0    4
57.0    3
58.0    3
55.0    3
52.0    3
64.0    3
50.0    3
46.0    3
51.0    3
37.0    3
54.0    2
59.0    2
48.0    2
61.0    2
38.0    2
31.0    2
60.0    1
33.0    1
62.0    1
Name: count, dtype: int64


## Column Q78: Student's code

In [20]:
cali_df['Q78'] = cali_df['Q78'].str.strip().str.upper()
replace_dict = {
    '(00008909547) LESLIE PATRICIA POLANCO VELASCO': '00008909547-JAVCALI',
    'JavCali(00008909547)'  : '00008909547-JAVCALI',
    '0000890954'            : '00008909547-JAVCALI',
    'JAVCALI(00008909547)'  :'00008909547-JAVCALI',
    'JAVCALI(00008909547)XIOMARA AMU':'00008909547-JAVCALI',
    'JAVCALI00008909547'    :'00008909547-JAVCALI',
    '00008909547'           :'00008909547-JAVCALI',
    'JAVCALI(00008909547'   :'00008909547-JAVCALI',
    'JAVCALI(00008909547)'  :'00008909547-JAVCALI',
    '8968160JAVECALI'   : '00008968160-JAVCALI',
    'JAVCALI 8959170'   : '00008959170-JAVCALI',
    'JAVCALI(8960388)'  : '00008960388-JAVCALI',
    'JAVCALI- (8959446)': '00008959446-JAVCALI',
    'JAVCALI8958506'    : '00008958506-JAVCALI',
    'JAVCALI8959955'    : '00008959955-JAVCALI',
    'JAVCALI8963141'    : '00008963141-JAVCALI',
    'JAVECALI8953122'   : '00008953122-JAVCALI',
    'JAVECALI8958190'   : '00008958190-JAVCALI',
    '8948180'           : '00008948180-JAVCALI',
    '8958113'           : '00008958113-JAVCALI',
    '300000121840'         : '30000121840-USBCALI',
    '30000085639'          : '30000085639-USBCALI',
    '30000116835'          : '30000116835-USBCALI',
    '30000116835 USB CALI' : '30000116835-USBCALI',
    '30000121119'          : '30000121119-USBCALI',
    '30000121119USBCALI'   : '30000121119-USBCALI',
    '30000121234 USBCALI'  : '30000121234-USBCALI',
    '30000121417USBCALI'   : '30000121417-USBCALI',
    '30000122711USB'       : '30000122711-USBCALI',
    '30000124761USBCALI'   : '30000124761-USBCALI',
    '3000122700USBCALI'    : '30000122700-USBCALI',
    '3000124761USBCALI'    : '30000124761-USBCALI',
    'USB-CALI-30000122709' : '30000122709-USBCALI',
    'USB-CALI-30000124779' : '30000124779-USBCALI',
    '3000000121840'        : '30000121840-USBCALI',
    'USB30000124779'       : '30000124779-USBCALI',
}
cali_df['Q78'] = cali_df['Q78'].replace(replace_dict)

C:\Users\hamso\AppData\Local\Temp\ipykernel_22136\1508596883.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hamso\AppData\Local\Temp\ipykernel_22136\1508596883.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Q78
00008909547-JAVCALI    22
00008948180-JAVCALI     2
00008953122-JAVCALI     5
00008958113-JAVCALI     1
00008958190-JAVCALI     7
00008958506-JAVCALI     3
00008959446-JAVCALI     2
00008959955-JAVCALI     1
00008960388-JAVCALI     1
00008963141-JAVCALI     1
00008968160-JAVCALI     1
000346984               3
30000085639-USBCALI     1
30000116835-USBCALI     2
30000121119-USBCALI     3
30000121417-USBCALI     4
30000121840-USBCALI     4
30000122700-USBCALI     1
30000122709-USBCALI     1
30000122711-USBCALI     2
30000124761-USBCALI     1
30000124779-USBCALI     1
Name: count, dtype: int64

In [21]:
temp_value_counts(cali_df, 'Q78')
create_bar_chart(cali_df, 'Q78', 'code', 'Code', output_folders[1])

Q78
00008909547-JAVCALI    22
00008958190-JAVCALI     7
00008953122-JAVCALI     5
30000121840-USBCALI     4
30000121417-USBCALI     4
000346984               3
00008958506-JAVCALI     3
30000121119-USBCALI     3
30000116835-USBCALI     2
00008959446-JAVCALI     2
30000122711-USBCALI     2
00008948180-JAVCALI     2
30000124779-USBCALI     1
30000124761-USBCALI     1
30000122709-USBCALI     1
00008960388-JAVCALI     1
00008959955-JAVCALI     1
00008968160-JAVCALI     1
00008963141-JAVCALI     1
30000122700-USBCALI     1
30000085639-USBCALI     1
00008958113-JAVCALI     1
Name: count, dtype: int64


In [24]:
cali_df.columns

Index(['ResponseId', 'LocationLatitude', 'LocationLongitude', 'Sex', 'Age',
       'EDuc', 'Etnia', 'Q69', 'Q70', 'Identity Fusion_1',
       ...
       'Q78_2', 'Q78_3', 'Q78_4', 'Deshum_1', 'Deshum_2', 'Deshum_3', 'Q63',
       'Q64', 'Q64_5_TEXT', 'Q78'],
      dtype='object', length=140)

# Georeferencing the dataset

In [25]:
def print_map(df, save_name):
    # Remove the rows with missing values in the column 'LocationLatitude'
    df = df.dropna(subset=['LocationLatitude'])

    # Create a folium map
    m = folium.Map(location=[4.570868, -74.082125], zoom_start=6, projection="mercator")

    # Add observations to the map
    for i in range(len(df)):
        # Create a marker with the latitude and longitude of the observation
        marker = folium.Marker([df.iloc[i, 1], df.iloc[i, 2]], popup=df.iloc[i, 3])
        # Add the marker to the map
        m.add_child(marker)

    # Save the map as an HTML file
    m.save(output_folders[1].joinpath(save_name + '.html'))


In [26]:
print_map(cali_df, 'cali_map')
print_map(df, 'colombia_map')

In [27]:
codigos = cali_df['Q78'].value_counts().sum()
total = cali_df.shape[0]
print(f'Porcentaje de personas que asignaron codigo: {100*codigos/total}%')

Porcentaje de personas que asignaron codigo: 58.47457627118644%
